In [4]:
# import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import sklearn
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
#
import os
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
#
from sklearn.preprocessing import LabelEncoder
make_lb = LabelEncoder()
#
from sklearn.preprocessing import LabelBinarizer
lbb = LabelBinarizer()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import RFECV

In [2]:
with open('df_trimmed.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df_final = pickle.load(f)
df_final.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,default payment next month,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,PAY_0N1,PAY_0N2
0,2873.0,350000.0,1.0,1.0,2.0,37.0,0,0.0,0.001046,0.001046,0.0,0.000000,0.0,0,0
1,3598.0,50000.0,2.0,2.0,1.0,37.0,2,0.0,0.542003,0.026267,0.0,0.000000,0.0,0,0
2,27623.0,50000.0,2.0,1.0,2.0,23.0,0,0.0,0.078257,0.077063,2.0,0.011310,0.0,1,1
3,6874.0,20000.0,1.0,3.0,1.0,56.0,0,0.0,0.639975,0.041567,0.0,0.000000,1.0,0,0
4,6444.0,110000.0,2.0,2.0,2.0,32.0,0,0.0,0.983186,0.041364,1.0,0.012245,0.0,0,0


In [3]:
df_final.drop(columns=['ID'], axis=1, inplace=True)

### General Layout

`rfecv = RFECV(estimator=MODEL, step=1, cv=,
              scoring='accuracy')`

In [5]:
df_final.rename(columns={'default payment next month':'default'}, inplace=True)
df_final.columns=df_final.columns.str.lower()
df_final.head()

,limit_bal,sex,education,marriage,age,pay_0,default,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,pay_0n1,pay_0n2
0,350000.0,1.0,1.0,2.0,37.0,0,0.0,0.001046,0.001046,0.0,0.000000,0.0,0,0
1,50000.0,2.0,2.0,1.0,37.0,2,0.0,0.542003,0.026267,0.0,0.000000,0.0,0,0
2,50000.0,2.0,1.0,2.0,23.0,0,0.0,0.078257,0.077063,2.0,0.011310,0.0,1,1
3,20000.0,1.0,3.0,1.0,56.0,0,0.0,0.639975,0.041567,0.0,0.000000,1.0,0,0
4,110000.0,2.0,2.0,2.0,32.0,0,0.0,0.983186,0.041364,1.0,0.012245,0.0,0,0


### I have to run my three base models with this data in order to see how it performs once I start to use a feature elimination module.


---

### Baseline Logistic Regression Model

In [6]:
# Preparing data for the model
df_fin_mms = df_final
df_fin_mms = minmax.fit_transform(df_fin_mms)

In [7]:
df_fin_mms = pd.DataFrame(df_fin_mms, columns=df_final.columns)
df_fin_mms.head()

,limit_bal,sex,education,marriage,age,pay_0,default,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,pay_0n1,pay_0n2
0,0.343434,0.0,0.166667,0.666667,0.275862,0.00,0.0,0.036158,0.000431,0.0,0.000000,0.0,0.0,0.0
1,0.040404,1.0,0.333333,0.333333,0.275862,0.25,0.0,0.133375,0.010819,0.0,0.000000,0.0,0.0,0.0
2,0.040404,1.0,0.166667,0.666667,0.034483,0.00,0.0,0.050034,0.031743,0.4,0.004299,0.0,1.0,1.0
3,0.010101,0.0,0.500000,0.333333,0.603448,0.00,0.0,0.150981,0.017122,0.0,0.000000,0.2,0.0,0.0
4,0.101010,1.0,0.333333,0.666667,0.189655,0.00,0.0,0.212660,0.017038,0.2,0.004655,0.0,0.0,0.0


In [8]:
y = df_fin_mms['default']
X = df_fin_mms.drop(columns=['default'], axis=1)

In [9]:
logreg_base = LogisticRegressionCV(random_state=42, cv=5, scoring = 'accuracy')
base_lr = logreg_base.fit(X, y)

In [14]:
cross_validate(logreg_base, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([1.43056703, 1.48034191, 1.44199896, 1.28604078, 1.38346505]),
 'score_time': array([0.01511312, 0.01458097, 0.01620579, 0.01441717, 0.013623  ]),
 'test_f1': array([0.46029777, 0.45083014, 0.47195122, 0.42637644, 0.48406863]),
 'test_recall': array([0.34934087, 0.33239171, 0.36440678, 0.31385485, 0.37229029]),
 'test_precision': array([0.67454545, 0.70039683, 0.66955017, 0.66467066, 0.69176883]),
 'test_accuracy': array([0.81878775, 0.82083333, 0.81958333, 0.81329444, 0.82454678])}

In [16]:
# Probably a better way of doing this
test_f1 = np.mean([0.46029777, 0.45083014, 0.47195122, 0.42637644, 0.48406863])
test_recall = np.mean([0.34934087, 0.33239171, 0.36440678, 0.31385485, 0.37229029])
test_precision = np.mean([0.67454545, 0.70039683, 0.66955017, 0.66467066, 0.69176883])
test_accuracy = np.mean([0.81878775, 0.82083333, 0.81958333, 0.81329444, 0.82454678])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.45870484000000006
test_recall: 0.3464569
test_precision: 0.680186388
test_accuracy: 0.819409126


---

### Baseline KNN Model

In [12]:
knn = KNeighborsClassifier()

In [13]:
cross_validate(knn, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([0.1059792 , 0.07364297, 0.07256985, 0.13773799, 0.07400608]),
 'score_time': array([1.84436679, 1.57892895, 1.60476208, 1.76483202, 1.61518502]),
 'test_f1': array([0.41520468, 0.42747188, 0.41743119, 0.41852286, 0.43367935]),
 'test_recall': array([0.33427495, 0.33992467, 0.34274953, 0.33647502, 0.35438266]),
 'test_precision': array([0.54783951, 0.57575758, 0.53372434, 0.55348837, 0.55869242]),
 'test_accuracy': array([0.79171006, 0.79854167, 0.78833333, 0.79329027, 0.79537404])}

In [15]:
# Probably a better way of doing this
test_f1 = np.mean([0.41520468, 0.42747188, 0.41743119, 0.41852286, 0.43367935])
test_recall = np.mean([0.33427495, 0.33992467, 0.34274953, 0.33647502, 0.35438266])
test_precision = np.mean([0.54783951, 0.57575758, 0.53372434, 0.55348837, 0.55869242])
test_accuracy = np.mean([0.79171006, 0.79854167, 0.78833333, 0.79329027, 0.79537404])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.422461992
test_recall: 0.341561366
test_precision: 0.5539004439999999
test_accuracy: 0.793449874


---

### DecisionTreeClassifier Base Model

In [17]:
dt_base = DecisionTreeClassifier(random_state=42)

In [18]:
cross_validate(dt_base, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([0.1804781 , 0.13114905, 0.12765884, 0.17276382, 0.13921213]),
 'score_time': array([0.01597786, 0.01603675, 0.01904011, 0.02118206, 0.01590109]),
 'test_f1': array([0.37683824, 0.38942976, 0.406165  , 0.38906606, 0.43054935]),
 'test_recall': array([0.38606403, 0.39548023, 0.42184557, 0.40245052, 0.45428841]),
 'test_precision': array([0.36804309, 0.38356164, 0.39160839, 0.37654321, 0.40916808]),
 'test_accuracy': array([0.71755884, 0.725625  , 0.72708333, 0.72056678, 0.73431965])}

In [19]:
test_f1 = np.mean([0.37683824, 0.38942976, 0.406165  , 0.38906606, 0.43054935])
test_recall = np.mean([0.38606403, 0.39548023, 0.42184557, 0.40245052, 0.45428841])
test_precision = np.mean([0.36804309, 0.38356164, 0.39160839, 0.37654321, 0.40916808])
test_accuracy = np.mean([0.71755884, 0.725625  , 0.72708333, 0.72056678, 0.73431965])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.39840968199999993
test_recall: 0.4120257519999999
test_precision: 0.38578488199999994
test_accuracy: 0.72503072


---

### Time to check out RFECV


In [20]:
# KNN doesn't support this
rfecv_lg = RFECV(estimator=logreg_base, step=1, min_features_to_select=10, cv=5,
              scoring='accuracy')
rfecv_dt = RFECV(estimator=dt_base, step=1, min_features_to_select=10, cv=5,
              scoring='accuracy')

In [21]:
rfecv_lg.fit(X,y)
rfecv_dt.fit(X,y)

RFECV(cv=5,
      estimator=DecisionTreeClassifier(class_weight=None, criterion='gini',
                                       max_depth=None, max_features=None,
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort=False, random_state=42,
                                       splitter='best'),
      min_features_to_select=10, n_jobs=None, scoring='accuracy', step=1,
      verbose=0)

In [22]:
lg_feats = pd.DataFrame(rfecv_lg.ranking_)
dt_feats = pd.DataFrame(rfecv_dt.ranking_)

In [23]:
# Generating the feature selection for LogReg
lg_feats = lg_feats.T
lg_feats.rename(columns={0:X.columns[0], 1:X.columns[1], 2:X.columns[2], 3:X.columns[3], 4:X.columns[4],
                         5:X.columns[5], 6:X.columns[6], 7:X.columns[7], 8:X.columns[8], 9:X.columns[9],
                         10:X.columns[10], 11:X.columns[11], 12:X.columns[12]}, inplace=True)

In [24]:
lg_feats = lg_feats.T
lg_feats.rename(columns={0:'Feature Ranks'}, inplace=True)
lg_feats

,Feature Ranks
limit_bal,1
sex,1
education,1
marriage,1
age,1
pay_0,1
avg_utilization,1
avg_payment_impact,1
debt_streak,1
debt_avg_delta,2


In [25]:
# Generating the feature selection for DecTree
dt_feats = dt_feats.T
dt_feats.rename(columns={0:X.columns[0], 1:X.columns[1], 2:X.columns[2], 3:X.columns[3], 4:X.columns[4],
                         5:X.columns[5], 6:X.columns[6], 7:X.columns[7], 8:X.columns[8], 9:X.columns[9],
                         10:X.columns[10], 11:X.columns[11], 12:X.columns[12]}, inplace=True)

In [26]:
dt_feats = dt_feats.T
dt_feats.rename(columns={0:'Feature Ranks'}, inplace=True)
dt_feats

,Feature Ranks
limit_bal,1
sex,1
education,1
marriage,1
age,1
pay_0,1
avg_utilization,1
avg_payment_impact,1
debt_streak,1
debt_avg_delta,1


### 1 means important, with larger numbers indicating less importance


---

## Model Refinement | Hyperparameter Tuning